In [ ]:
# This script removes extra parts in the sp5 structure that are beyond the image frame.

In [2]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from skimage.transform import estimate_transform

In [6]:
stack = 'MD594'
first_bs_section, last_bs_section = section_range_lookup[stack]

In [10]:
from shapely.geometry import Polygon, LineString

In [151]:
# delete all latest labelings
for sec in range(first_bs_section, last_bs_section+1):
    
    dm = DataManager(stack=stack, section=sec, labeling_dir='/home/yuncong/CSHL_data_labelings_losslessAlignCropped/')
    res = dm.load_review_result_path('yuncong', 'latest', suffix='consolidated')
    if res is None:
        continue
        
    fn, usr, ts = res
    os.system('rm %s' % fn)

In [108]:
from scipy.spatial.distance import cdist

In [152]:
borders = LineString([(dm.image_width-1, 0), (dm.image_width-1, dm.image_height-1), (0, dm.image_height-1)])

for sec in range(first_bs_section, last_bs_section+1):
# for sec in range(190, 191):

    try:
        dm = DataManager(stack=stack, section=sec,
                         labeling_dir='/home/yuncong/CSHL_data_labelings_losslessAlignCropped/')
        user, ts, _, res  = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    except:
        continue
    
    new_res = []
    for pol in res:
        new_pol = pol.copy()
        
        vs = np.array(pol['vertices'])
        
        if len(vs) == 0:
            continue

        if len(np.where((vs[:,0] < 0) | (vs[:,0] >= dm.image_width) | \
                        (vs[:,1] < 0) | (vs[:,1] >= dm.image_height))[0]) > 0:
            intersections = Polygon(vs).exterior.intersection(borders)
            if len(intersections) > 2:
                sys.stderr.write('more than two intersections with the border: %d\n' % sec)
                continue
                
            e1, e2 = [(int(g.x), int(g.y)) for g in intersections.geoms]
            w = np.where((vs[:,0] >= dm.image_width) | (vs[:,1] >= dm.image_height))[0]
#             print w
            w1 = w.min()
            w2 = w.max()
            
            valid_e1 = vs[w1-1]
            valid_e2 = vs[w2+1]
            
            if e1[0] == e2[0]: # intersect on right
                if (e1[1] - e2[1]) * (valid_e1[1] - valid_e2[1]) > 0:
                    vs = np.vstack([vs[:w1], e1, e2, vs[w2+1:]])
                else:
                    vs = np.vstack([vs[:w1], e2, e1, vs[w2+1:]])
            elif e1[1] == e2[1]: # intersect on bottom
                if (e1[0] - e2[0]) * (valid_e1[0] - valid_e2[0]) > 0:
                    vs = np.vstack([vs[:w1], e1, e2, vs[w2+1:]])
                else:
                    vs = np.vstack([vs[:w1], e2, e1, vs[w2+1:]])
            elif: # one on right, one on bottom
                d = cdist([valid_e1, valid_e2], [e1, e2])
                r, c = np.unravel_index(np.argmin(d), (2,2))
                if r == c:
                    vs = np.vstack([vs[:w1], e1, (dm.image_width-1, dm.image_height-1), e2, vs[w2+1:]])
                else:
                    vs = np.vstack([vs[:w1], e2, (dm.image_width-1, dm.image_height-1), e1, vs[w2+1:]])
                         
        new_pol['vertices'] = vs
        new_pol['labelPos'] = np.atleast_2d(vs).mean(axis=0)
        
        new_res.append(new_pol)
        
    out_dir = create_if_not_exists('/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped/' + \
                                   stack + '/' + '%04d'%sec)

    new_ts = datetime.datetime.now().strftime("%m%d%Y%H%M%S")
    pickle.dump(new_res, 
                open(out_dir + '/' + stack + '_' + '%04d'%sec + '_' + user + '_' + new_ts + '_consolidated.pkl', 'w'))

username yuncong does not have any annotations for current section 93 
username yuncong does not have any annotations for current section 94 
username yuncong does not have any annotations for current section 95 
username yuncong does not have any annotations for current section 97 
username yuncong does not have any annotations for current section 99 
username yuncong does not have any annotations for current section 101 
username yuncong does not have any annotations for current section 103 
username yuncong does not have any annotations for current section 105 
username yuncong does not have any annotations for current section 107 
username yuncong does not have any annotations for current section 109 
username yuncong does not have any annotations for current section 111 
username yuncong does not have any annotations for current section 113 
username yuncong does not have any annotations for current section 115 
username yuncong does not have any annotations for current section 11